Low amount of data and high number of features, so we need a classification model that has bias/low variance such as Linear SVM.

https://www.kdnuggets.com/2020/05/guide-choose-right-machine-learning-algorithm.html

In [1]:
import os
os.chdir('..')
os.chdir('../resources')
pwd = os.getcwd()

In [2]:
%%time
import pandas as pd
import cv2

features = []
pixels = []
img_names = []
feature_names = ['head', 'spine', 'leg', 'arm', 'hand', 'foot']
for feature in feature_names+ ['none']:
    file_names = os.listdir(pwd+'/features/'+feature)
    img_names = img_names + file_names
    for file in file_names:
        features.append(feature)
        pixels.append(cv2.imread(pwd+'/features/'+feature+'/'+file, 0))

feature_images = pd.DataFrame({'image_name': img_names,'class' : features, 'image_pixels': pixels})

print('Total features: ', feature_images.shape[0])
print(feature_images.value_counts('class'))
feature_images

Total features:  2457
class
head     1365
none      443
spine     410
hand      117
arm        75
leg        29
foot       18
dtype: int64
Wall time: 1.96 s


,image_name,class,image_pixels
0,19150_2021-01-24_10.31_103052326.png,head,"[[79, 78, 82, 85, 87, 89, 93, 97, 100, 104, 10..."
1,19150_2021-01-24_19.28_191920932.png,head,"[[95, 96, 98, 100, 101, 104, 108, 108, 106, 10..."
2,19150_2021-01-24_19.28_192010654.png,head,"[[103, 101, 99, 92, 83, 83, 90, 93, 96, 97, 94..."
3,19150_2021-01-24_19.28_192723415.png,head,"[[96, 92, 88, 81, 84, 86, 83, 84, 82, 83, 83, ..."
4,19150_2021-01-24_19.28_192733399.png,head,"[[124, 131, 153, 171, 177, 153, 111, 102, 115,..."
...,...,...,...
2452,21228_2021-04-14_10.51_103812596.png,none,"[[157, 153, 153, 145, 135, 139, 132, 126, 129,..."
2453,21252_2021-04-24_16.53_165224510.png,none,"[[155, 143, 128, 117, 110, 111, 108, 106, 106,..."
2454,21262_2021-04-20_21.59_215422790.png,none,"[[121, 119, 116, 111, 108, 107, 106, 107, 107,..."
2455,21273_2021-04-21_14.58_145020212.png,none,"[[89, 90, 92, 93, 95, 96, 98, 98, 101, 103, 10..."


In [3]:
def image_size(row):
    row['width'] = row['image_pixels'].shape[0]
    row['height'] = row['image_pixels'].shape[1]
    return row

f = feature_images.apply(image_size, axis=1)
feature_images = f[f['width']>10]
feature_images

,image_name,class,image_pixels,width,height
0,19150_2021-01-24_10.31_103052326.png,head,"[[79, 78, 82, 85, 87, 89, 93, 97, 100, 104, 10...",145,145
1,19150_2021-01-24_19.28_191920932.png,head,"[[95, 96, 98, 100, 101, 104, 108, 108, 106, 10...",154,154
2,19150_2021-01-24_19.28_192010654.png,head,"[[103, 101, 99, 92, 83, 83, 90, 93, 96, 97, 94...",140,140
3,19150_2021-01-24_19.28_192723415.png,head,"[[96, 92, 88, 81, 84, 86, 83, 84, 82, 83, 83, ...",155,155
4,19150_2021-01-24_19.28_192733399.png,head,"[[124, 131, 153, 171, 177, 153, 111, 102, 115,...",152,152
...,...,...,...,...,...
2452,21228_2021-04-14_10.51_103812596.png,none,"[[157, 153, 153, 145, 135, 139, 132, 126, 129,...",96,96
2453,21252_2021-04-24_16.53_165224510.png,none,"[[155, 143, 128, 117, 110, 111, 108, 106, 106,...",145,145
2454,21262_2021-04-20_21.59_215422790.png,none,"[[121, 119, 116, 111, 108, 107, 106, 107, 107,...",107,107
2455,21273_2021-04-21_14.58_145020212.png,none,"[[89, 90, 92, 93, 95, 96, 98, 98, 101, 103, 10...",191,191


Get images ready for HOG:
1. Filter
2. Resize to (128, 128)

In [4]:
%%time
from Process import process
def resize(x):
    return cv2.resize(x,(128, 128))

feature_images['filtered_pixels'] = feature_images['image_pixels'].apply(process, crop_thresh=0)
feature_images['resized_pixels'] = feature_images['filtered_pixels'].apply(resize)
feature_images

<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 46.4 s


,image_name,class,image_pixels,width,height,filtered_pixels,resized_pixels
0,19150_2021-01-24_10.31_103052326.png,head,"[[79, 78, 82, 85, 87, 89, 93, 97, 100, 104, 10...",145,145,"[[0, 0, 24, 23, 23, 24, 24, 25, 33, 43, 49, 60...","[[2, 6, 24, 23, 24, 24, 25, 33, 43, 52, 61, 68..."
1,19150_2021-01-24_19.28_191920932.png,head,"[[95, 96, 98, 100, 101, 104, 108, 108, 106, 10...",154,154,"[[0, 0, 73, 73, 73, 75, 75, 81, 85, 97, 97, 10...","[[7, 27, 73, 73, 74, 76, 82, 91, 97, 100, 105,..."
2,19150_2021-01-24_19.28_192010654.png,head,"[[103, 101, 99, 92, 83, 83, 90, 93, 96, 97, 94...",140,140,"[[0, 0, 76, 73, 73, 73, 73, 73, 73, 73, 73, 73...","[[4, 14, 75, 73, 73, 73, 73, 73, 73, 73, 73, 7..."
3,19150_2021-01-24_19.28_192723415.png,head,"[[96, 92, 88, 81, 84, 86, 83, 84, 82, 83, 83, ...",155,155,"[[0, 0, 50, 50, 50, 50, 49, 48, 49, 49, 50, 52...","[[5, 19, 50, 50, 50, 49, 48, 49, 50, 52, 56, 5..."
4,19150_2021-01-24_19.28_192733399.png,head,"[[124, 131, 153, 171, 177, 153, 111, 102, 115,...",152,152,"[[0, 0, 123, 136, 136, 136, 121, 121, 117, 103...","[[11, 43, 129, 136, 136, 121, 120, 111, 99, 93..."
...,...,...,...,...,...,...,...
2452,21228_2021-04-14_10.51_103812596.png,none,"[[157, 153, 153, 145, 135, 139, 132, 126, 129,...",96,96,"[[0, 0, 239, 233, 233, 228, 205, 205, 186, 175...","[[0, 0, 89, 238, 234, 233, 231, 225, 208, 205,..."
2453,21252_2021-04-24_16.53_165224510.png,none,"[[155, 143, 128, 117, 110, 111, 108, 106, 106,...",145,145,"[[0, 0, 146, 148, 143, 141, 136, 132, 129, 129...","[[10, 37, 146, 145, 142, 137, 132, 129, 129, 1..."
2454,21262_2021-04-20_21.59_215422790.png,none,"[[121, 119, 116, 111, 108, 107, 106, 107, 107,...",107,107,"[[0, 0, 158, 158, 158, 158, 156, 156, 142, 135...","[[0, 0, 93, 158, 158, 158, 158, 156, 156, 150,..."
2455,21273_2021-04-21_14.58_145020212.png,none,"[[89, 90, 92, 93, 95, 96, 98, 98, 101, 103, 10...",191,191,"[[0, 0, 42, 44, 44, 45, 47, 49, 50, 53, 53, 53...","[[10, 34, 44, 45, 47, 49, 53, 53, 53, 53, 54, ..."


Compute HOG features

In [5]:
%%time
from Hog import compute_hog
feature_images['hog'] = feature_images['resized_pixels'].apply(compute_hog)
feature_images

<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 7.27 s


,image_name,class,image_pixels,width,height,filtered_pixels,resized_pixels,hog
0,19150_2021-01-24_10.31_103052326.png,head,"[[79, 78, 82, 85, 87, 89, 93, 97, 100, 104, 10...",145,145,"[[0, 0, 24, 23, 23, 24, 24, 25, 33, 43, 49, 60...","[[2, 6, 24, 23, 24, 24, 25, 33, 43, 52, 61, 68...","[[0.06334161555010934, 0.06259491313097862, 0...."
1,19150_2021-01-24_19.28_191920932.png,head,"[[95, 96, 98, 100, 101, 104, 108, 108, 106, 10...",154,154,"[[0, 0, 73, 73, 73, 75, 75, 81, 85, 97, 97, 10...","[[7, 27, 73, 73, 74, 76, 82, 91, 97, 100, 105,...","[[0.0559627705081274, 0.005103732810678231, 0...."
2,19150_2021-01-24_19.28_192010654.png,head,"[[103, 101, 99, 92, 83, 83, 90, 93, 96, 97, 94...",140,140,"[[0, 0, 76, 73, 73, 73, 73, 73, 73, 73, 73, 73...","[[4, 14, 75, 73, 73, 73, 73, 73, 73, 73, 73, 7...","[[0.07258414365917265, 0.01485513539255349, 0...."
3,19150_2021-01-24_19.28_192723415.png,head,"[[96, 92, 88, 81, 84, 86, 83, 84, 82, 83, 83, ...",155,155,"[[0, 0, 50, 50, 50, 50, 49, 48, 49, 49, 50, 52...","[[5, 19, 50, 50, 50, 49, 48, 49, 50, 52, 56, 5...","[[0.06225310740135644, 0.017496744922557792, 0..."
4,19150_2021-01-24_19.28_192733399.png,head,"[[124, 131, 153, 171, 177, 153, 111, 102, 115,...",152,152,"[[0, 0, 123, 136, 136, 136, 121, 121, 117, 103...","[[11, 43, 129, 136, 136, 121, 120, 111, 99, 93...","[[0.22807042624161517, 0.17636213754592314, 0...."
...,...,...,...,...,...,...,...,...
2452,21228_2021-04-14_10.51_103812596.png,none,"[[157, 153, 153, 145, 135, 139, 132, 126, 129,...",96,96,"[[0, 0, 239, 233, 233, 228, 205, 205, 186, 175...","[[0, 0, 89, 238, 234, 233, 231, 225, 208, 205,...","[[0.21738151041267945, 0.04581817159322388, 0...."
2453,21252_2021-04-24_16.53_165224510.png,none,"[[155, 143, 128, 117, 110, 111, 108, 106, 106,...",145,145,"[[0, 0, 146, 148, 143, 141, 136, 132, 129, 129...","[[10, 37, 146, 145, 142, 137, 132, 129, 129, 1...","[[0.11641136360850891, 0.036461367914496716, 0..."
2454,21262_2021-04-20_21.59_215422790.png,none,"[[121, 119, 116, 111, 108, 107, 106, 107, 107,...",107,107,"[[0, 0, 158, 158, 158, 158, 156, 156, 142, 135...","[[0, 0, 93, 158, 158, 158, 158, 156, 156, 150,...","[[0.11611217651214777, 0.023683375978650632, 0..."
2455,21273_2021-04-21_14.58_145020212.png,none,"[[89, 90, 92, 93, 95, 96, 98, 98, 101, 103, 10...",191,191,"[[0, 0, 42, 44, 44, 45, 47, 49, 50, 53, 53, 53...","[[10, 34, 44, 45, 47, 49, 53, 53, 53, 53, 54, ...","[[0.031094257510773278, 0.007472437138147385, ..."


Separate into test and train data

Train Linear SVMs for each feature

In [6]:
%%time
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
feature_svm = []
fully_trained = []
accuracy = []
classification_rep = []
for feature in feature_names:
    df = feature_images[feature_images['class'].isin([feature, 'none'])]
    
    labels =  np.array(df['class']).reshape(len(df['class']),1)
    hog_features = np.stack(df['hog'], axis=1)[0]
    data = np.hstack((hog_features,labels))

    np.random.shuffle(data)
    percentage = 80
    partition = int(len(data)*percentage/100)

    x_train, x_test = data[:partition,:-1],  data[partition:,:-1]
    y_train, y_test = data[:partition,-1:].ravel() , data[partition:,-1:].ravel()

    clf = svm.SVC(probability=True)
    clf.fit(x_train,y_train)
    
    clf2 = svm.SVC(probability=True)
    clf2.fit(data[:len(data),:-1],data[:len(data),-1:].ravel())
    
    y_pred = clf.predict(x_test)
    
    fully_trained.append(clf2)
    feature_svm.append(clf)
    accuracy.append(accuracy_score(y_test, y_pred))
    classification_rep.append(classification_report(y_test, y_pred))

results = pd.DataFrame({'feature': feature_names, 'svm': feature_svm, 'accuracy': accuracy, 'classification report': classification_rep})

C:\Users\emery\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emery\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emery\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emery\anaconda3\lib\site-packages\sklearn\metr

Wall time: 23.7 s


C:\Users\emery\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emery\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emery\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Print results

In [7]:
for idx,row in results.iterrows():
    print(row['feature'])
    print('accuracy: '+str(row['accuracy']))
    print(row['classification report'])
    print('\n')

head
accuracy: 0.9779005524861878
              precision    recall  f1-score   support

        head       0.98      0.99      0.99       265
        none       0.98      0.94      0.96        97

    accuracy                           0.98       362
   macro avg       0.98      0.97      0.97       362
weighted avg       0.98      0.98      0.98       362



spine
accuracy: 0.9122807017543859
              precision    recall  f1-score   support

        none       0.91      0.94      0.92        94
       spine       0.92      0.88      0.90        77

    accuracy                           0.91       171
   macro avg       0.91      0.91      0.91       171
weighted avg       0.91      0.91      0.91       171



leg
accuracy: 0.9368421052631579
              precision    recall  f1-score   support

         leg       0.00      0.00      0.00         6
        none       0.94      1.00      0.97        89

    accuracy                           0.94        95
   macro avg       0.4

In [8]:
import pickle
from pathlib import Path

svm_folder =  pwd+"/SVM/"
for clf in fully_trained:
    class_ = clf.classes_
    class_ = np.copy(class_).tolist()
    class_.remove('none')
    pkl_filename = class_[0]+'.pkl'
    with open(svm_folder+pkl_filename, 'wb+') as file:
        pickle.dump(clf, file)